In [1]:
import os
os.environ['SPARK_HOME']="/opt/cloudera/parcels/SPARK2/lib/spark2"

In [2]:
import findspark

In [3]:
findspark.init()

In [4]:
import pyspark
import numpy as np

In [5]:
conf = pyspark.SparkConf().\
    setAppName('sentiment-analysis-Project4').\
    setMaster('local[2]')

In [6]:
from pyspark.sql import SQLContext, HiveContext
sc = pyspark.SparkContext(conf=conf)
sqlContext = HiveContext(sc)

In [7]:
from pyspark.sql import functions as fn

In [8]:
from pyspark.ml.feature import Tokenizer

In [9]:
from pyspark.ml.feature import CountVectorizer

In [10]:
from pyspark.ml import Pipeline

In [11]:
!wget https://github.com/majedab/IST718Project/blob/master/dfFineFood_2.csv?raw=true -O dfFineFood_copy.csv

--2016-11-30 11:21:56--  https://github.com/majedab/IST718Project/blob/master/dfFineFood_2.csv?raw=true
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/majedab/IST718Project/raw/master/dfFineFood_2.csv [following]
--2016-11-30 11:21:56--  https://github.com/majedab/IST718Project/raw/master/dfFineFood_2.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/majedab/IST718Project/master/dfFineFood_2.csv [following]
--2016-11-30 11:21:56--  https://media.githubusercontent.com/media/majedab/IST718Project/master/dfFineFood_2.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.116.133
Connecting to media.githubusercontent.com (media.githubusercontent.com)|151.101.116.133|:443... connected.
HTTP re

In [12]:
fineFood_df = sqlContext.read\
    .format("com.databricks.spark.csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .option("nullValue", 'NA')\
    .load("file:///home/maalbarr/Project/dfFineFood_copy.csv")

In [17]:
!wget https://github.com/daniel-acuna/python_data_science_intro/blob/master/data/sentiments.parquet.zip?raw=true -O sentiments.parquet.zip && unzip sentiments.parquet.zip && rm sentiments.parquet.zip

--2016-11-30 00:03:23--  https://github.com/daniel-acuna/python_data_science_intro/blob/master/data/sentiments.parquet.zip?raw=true
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/daniel-acuna/python_data_science_intro/raw/master/data/sentiments.parquet.zip [following]
--2016-11-30 00:03:23--  https://github.com/daniel-acuna/python_data_science_intro/raw/master/data/sentiments.parquet.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/daniel-acuna/python_data_science_intro/master/data/sentiments.parquet.zip [following]
--2016-11-30 00:03:23--  https://raw.githubusercontent.com/daniel-acuna/python_data_science_intro/master/data/sentiments.parquet.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.11

In [13]:
sentiments_df = sqlContext.read.load("file:///home/maalbarr/Project/sentiments.parquet")

In [14]:
import requests
stop_words = requests.get('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words').text.split()

In [15]:
fineFood_df.show(1)

+---+-------+---------+------------+-----+--------------------+--------------------+-------------------+
| ID|   Time|IsWeekday|IsHolidayUSA|Score|             Summary|                Text|HelpfulnessFraction|
+---+-------+---------+------------+-----+--------------------+--------------------+-------------------+
|  0|10/8/99|        1|           0|    1|every book is edu...|this witty little...|                  0|
+---+-------+---------+------------+-----+--------------------+--------------------+-------------------+
only showing top 1 row



In [16]:
fineFood_df.where(fn.col('Score') == 1).first()

Row(ID=0, Time='10/8/99', IsWeekday=1, IsHolidayUSA=0, Score=1, Summary='every book is educational', Text="this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, india, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will still be able to recite from memory when he is  in college", HelpfulnessFraction='0')

In [17]:
from pyspark.ml.feature import RegexTokenizer
tokenizer = RegexTokenizer().setGaps(False)\
  .setPattern("\\p{L}+")\
  .setInputCol("Text")\
  .setOutputCol("words")

In [20]:
from pyspark.ml.feature import StopWordsRemover
sw_filter = StopWordsRemover()\
  .setStopWords(stop_words)\
  .setCaseSensitive(False)\
  .setInputCol("words")\
  .setOutputCol("filtered")

In [21]:
from pyspark.ml.feature import CountVectorizer

# we will remove words that appear in 5 docs or less
cv = CountVectorizer(minTF=1., minDF=5., vocabSize=2**17)\
  .setInputCol("filtered")\
  .setOutputCol("tf")

In [22]:
# we now create a pipelined transformer
cv_pipeline = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(fineFood_df)

In [24]:
# now we can make the transformation between the raw text and the counts
cv_pipeline.transform(fineFood_df).show(5)

+---+--------+---------+------------+-----+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
| ID|    Time|IsWeekday|IsHolidayUSA|Score|             Summary|                Text|HelpfulnessFraction|               words|            filtered|                  tf|
+---+--------+---------+------------+-----+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|  0| 10/8/99|        1|           0|    1|every book is edu...|this witty little...|                  0|[this, witty, lit...|[witty, little, b...|(36579,[2,12,20,6...|
|  1|10/25/99|        1|           0|    1|this whole series...|i can remember se...|                  1|[i, can, remember...|[remember, seeing...|(36579,[25,35,37,...|
|  2| 12/2/99|        1|           0|    1|entertainingl funny!|beetlejuice is a ...|                  0|[beetlejuice, is,...|[beetlejuice, wri...|(36579,[

In [25]:
from pyspark.ml.feature import IDF
idf = IDF().\
    setInputCol('tf').\
    setOutputCol('tfidf')

In [27]:
idf_pipeline = Pipeline(stages=[cv_pipeline, idf]).fit(fineFood_df)

In [28]:
training_df, validation_df, testing_df = fineFood_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [29]:
[training_df.count(), validation_df.count(), testing_df.count()]

[340899, 170355, 57090]

In [31]:
from pyspark.ml.classification import LogisticRegression

In [32]:
lambda_par = 0.02
alpha_par = 0.3
en_lr = LogisticRegression().\
        setLabelCol('Score').\
        setFeaturesCol('tfidf').\
        setRegParam(lambda_par).\
        setMaxIter(100).\
        setElasticNetParam(alpha_par)

In [33]:
from pyspark.ml.tuning import ParamGridBuilder

In [34]:
en_lr_estimator = Pipeline(stages=[idf_pipeline, en_lr])

In [35]:
grid = ParamGridBuilder().\
    addGrid(en_lr.regParam, [0., 0.01, 0.02]).\
    addGrid(en_lr.elasticNetParam, [0., 0.2, 0.4]).\
    build()

In [86]:
grid

[{Param(parent='LogisticRegression_477f927862dad5f58878', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
  Param(parent='LogisticRegression_477f927862dad5f58878', name='regParam', doc='regularization parameter (>= 0).'): 0.0},
 {Param(parent='LogisticRegression_477f927862dad5f58878', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
  Param(parent='LogisticRegression_477f927862dad5f58878', name='regParam', doc='regularization parameter (>= 0).'): 0.01},
 {Param(parent='LogisticRegression_477f927862dad5f58878', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
  Param(parent='LogisticRegression_477f927862dad5f58878', name='regParam', do

In [36]:
all_models = []
for j in range(len(grid)):
    print("Fitting model {}".format(j+1))
    model = en_lr_estimator.fit(training_df, grid[j])
    all_models.append(model)

Fitting model 1
Fitting model 2
Fitting model 3
Fitting model 4
Fitting model 5
Fitting model 6
Fitting model 7
Fitting model 8
Fitting model 9


In [38]:
# estimate the accuracy of each of them:
accuracies = [m.\
    transform(validation_df).\
    select(fn.avg(fn.expr('Score = prediction').astype('float')).alias('accuracy')).\
    first().\
    accuracy for m in all_models]
import numpy as np
best_model_idx = np.argmax(accuracies)
best_model = all_models[best_model_idx]
accuracies[best_model_idx]

0.9171142613953215

In [44]:
best_model.transform(testing_df).select(fn.avg(fn.expr('float(prediction = Score)'))).show()

+--------------------------------+
|avg(float((prediction = score)))|
+--------------------------------+
|              0.9181993343843055|
+--------------------------------+



In [48]:
import pandas as pd
vocabulary = idf_pipeline.stages[0].stages[-1].vocabulary
en_weights = best_model.stages[-1].coefficients.toArray()
en_coeffs_df = pd.DataFrame({'word': vocabulary, 'weight': en_weights})

In [49]:
# Negative 
en_coeffs_df.sort_values('weight').head(5)

,weight,word
34276,-0.956534,rattlers
25679,-0.913749,ramada
31049,-0.842255,chatter
29920,-0.837219,coils
34711,-0.762865,distrustful


In [50]:
# Positive  
en_coeffs_df.sort_values('weight', ascending=False).head(5)

,weight,word
27243,1.056668,accuse
34110,1.001762,talcum
21628,0.835333,hiss
28287,0.802772,netrush
30953,0.782264,hiring


In [51]:
notHolidays_df = fineFood_df.where(fn.col('IsHolidayUSA') == 1)
notHolidays_df.show(1)

+---+--------+---------+------------+-----+------------+--------------------+-------------------+
| ID|    Time|IsWeekday|IsHolidayUSA|Score|     Summary|                Text|HelpfulnessFraction|
+---+--------+---------+------------+-----+------------+--------------------+-------------------+
| 94|10/31/02|        1|           1|    1|great comedy|beetlejuice is th...|                  1|
+---+--------+---------+------------+-----+------------+--------------------+-------------------+
only showing top 1 row



# Now we are building models on the dataset in which all reviews were done in holiday 

In [53]:
cv_pipeline_h = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(notHolidays_df)
idf_pipeline_h = Pipeline(stages=[cv_pipeline_h, idf]).fit(notHolidays_df)
training_df_h, validation_df_h, testing_df_h = notHolidays_df.randomSplit([0.6, 0.3, 0.1], seed=0)
en_lr_estimator_h = Pipeline(stages=[idf_pipeline_h, en_lr])
grid = ParamGridBuilder().\
    addGrid(en_lr.regParam, [0., 0.01, 0.02]).\
    addGrid(en_lr.elasticNetParam, [0., 0.2, 0.4]).\
    build()

In [54]:
all_models_h = []
for j in range(len(grid)):
    print("Fitting model {}".format(j+1))
    model = en_lr_estimator_h.fit(training_df_h, grid[j])
    all_models_h.append(model)

Fitting model 1
Fitting model 2
Fitting model 3
Fitting model 4
Fitting model 5
Fitting model 6
Fitting model 7
Fitting model 8
Fitting model 9


In [55]:
accuracies_h = [m.\
    transform(validation_df).\
    select(fn.avg(fn.expr('Score = prediction').astype('float')).alias('accuracy')).\
    first().\
    accuracy for m in all_models_h]
import numpy as np
best_model_h_idx = np.argmax(accuracies)
best_model_h = all_models[best_model_h_idx]
accuracies_h[best_model_h_idx]

0.9171142613953215

In [56]:
best_model_h.transform(testing_df_h).select(fn.avg(fn.expr('float(prediction = Score)'))).show()

+--------------------------------+
|avg(float((prediction = Score)))|
+--------------------------------+
|              0.9343434343434344|
+--------------------------------+



In [77]:
accuracies

[0.9089665698101025,
 0.9171142613953215,
 0.9157641395908543,
 0.9089665698101025,
 0.897924921487482,
 0.8847348184673183,
 0.9089665698101025,
 0.8863608347274808,
 0.8743682310469314]

In [78]:
accuracies_h

[0.8481406474714567,
 0.8679287370491033,
 0.8695195327404538,
 0.8481406474714567,
 0.8684981362449004,
 0.8700654515570426,
 0.8481406474714567,
 0.8695782336884741,
 0.8686448886149511]

# Building models on not holidays dataset 

In [81]:
notHolidays_df = fineFood_df.where(fn.col('IsHolidayUSA') == 0)
cv_pipeline_noth = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(notHolidays_df)
idf_pipeline_noth = Pipeline(stages=[cv_pipeline_h, idf]).fit(notHolidays_df)
training_df_noth, validation_df_noth, testing_df_noth = notHolidays_df.randomSplit([0.6, 0.3, 0.1], seed=0)
en_lr_estimator_noth = Pipeline(stages=[idf_pipeline_noth, en_lr])
grid = ParamGridBuilder().\
    addGrid(en_lr.regParam, [0., 0.01, 0.02]).\
    addGrid(en_lr.elasticNetParam, [0., 0.2, 0.4]).\
    build()

In [83]:
all_models_notholiday = []
for j in range(len(grid)):
    print("Fitting model {}".format(j+1))
    model = en_lr_estimator_noth.fit(training_df_noth, grid[j])
    all_models_notholiday.append(model)

Fitting model 1
Fitting model 2
Fitting model 3
Fitting model 4
Fitting model 5
Fitting model 6
Fitting model 7
Fitting model 8
Fitting model 9


In [84]:
accuracies_noth = [m.\
    transform(validation_df).\
    select(fn.avg(fn.expr('Score = prediction').astype('float')).alias('accuracy')).\
    first().\
    accuracy for m in all_models_notholiday]

In [85]:
accuracies_noth

[0.9115494115229961,
 0.9093892166358487,
 0.906800504828153,
 0.9115494115229961,
 0.8949663937072584,
 0.8845587156232573,
 0.9115494115229961,
 0.8862903935898565,
 0.8737929617563324]

In [95]:
best_model_noth_idx = np.argmax(accuracies_noth)
best_model_noth = all_models[best_model_h_idx]
grid[best_model_idx]

{Param(parent='LogisticRegression_477f927862dad5f58878', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_477f927862dad5f58878', name='regParam', doc='regularization parameter (>= 0).'): 0.01}

In [93]:
grid[best_model_h_idx]

{Param(parent='LogisticRegression_477f927862dad5f58878', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_477f927862dad5f58878', name='regParam', doc='regularization parameter (>= 0).'): 0.01}

In [96]:
grid[best_model_noth_idx]

{Param(parent='LogisticRegression_477f927862dad5f58878', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_477f927862dad5f58878', name='regParam', doc='regularization parameter (>= 0).'): 0.0}

# SVM attempt

We tried to apply SVM on this dataset but we could not fit the data 

In [119]:
idf_pipeline = Pipeline(stages=[cv_pipeline, idf]).fit(fineFood_df)
svmFineFood_df= idf_pipeline.transform(fineFood_df).select('tf','tfidf','HelpfulnessFraction','Score','IsHolidayUSA')

In [120]:
svm_training_df, svm_validation_df, svm_testing_df = svmFineFood_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [125]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
model= SVMWithSGD.train(svm_training_df['tfidf'], iterations=10)

TypeError: 'Column' object is not callable